### What is reified
In Kotlin we have generics. Generics in simple term means for a function, class we only supply the ``Type``

In [ ]:
open class Actor
class Hero: Actor()
class Heroine: Actor()


fun <T : Actor> getActor() {
    println("Actor is guessed ")
}

fun noReified() {
    getActor<Hero>()
    getActor<Heroine>()
}

In the above function, when we call this function, we cannot use the ``T`` or type parameter. What does that mean? Meaning once we call the function the type ``T`` is erased. For instance if we being to look upon the bytecode we will see the following

In [ ]:
  public static final void getActor() {
      String var0 = "Actor is guessed ";
      System.out.println(var0);
   }
 
 public static final void noReified() {
      getActor();
      getActor();
   }

When we call the function, though we passed ``Hero`` or ``Heroine``, we couldn't know the type of it. So in the bytecode we saw that the function  ``getActor()`` doesn't  know what type is being used to call the function. Meaning the caller ``noReified()`` knows the type in which it is being called but the function once being called doesn't know what is the type of it. If in cases we do want to know the passed type ``T``, reified will help us  

In [ ]:
inline fun <reified T : Actor> getActorReified() {
    println("Actor T is guessed = ${T::class.java}")
}

fun withReified() {
    getActorReified<Hero>()
    getActorReified<Heroine>()
}

Now if we see the bytecode of the same we will see

In [ ]:
public static final void getActorReified() {
      int $i$f$getActorReified = 0;
      StringBuilder var10000 = (new StringBuilder()).append("Actor T is guessed = ");
      Intrinsics.reifiedOperationMarker(4, "T");
      String var1 = var10000.append(Actor.class).toString();
      System.out.println(var1);
   }
// though the above reified function doesn't make sense, but the caller site will look interesting
public static final void withReified() {
      int $i$f$getActorReified = false;
      String var1 = "Actor T is guessed = " + Hero.class;
      System.out.println(var1);
      $i$f$getActorReified = false;
      var1 = "Actor T is guessed = " + Heroine.class;
      System.out.println(var1);
   }

If you look in the caller site, we see that the actual type parameter is retained there. Meaning we called with ``Hero`` and ``Heroine``, that information is now presented to the caller. This is the reason we needed ``inline``.  ``inline`` simply means that copy the code to the caller site. But when we do with ``reified``, it will not only copy but also allows us to add information about the type ``T`` to the caller site and paste the code along with the ``T`` type. If we didn't have ``inline``, then  it wouldn't be possible to maintain the type to the called function.  So due to this both of them go hand by hand.